Unsupervised learning in trading involves using ML techniques to analyse data and discover patterns, relationships, and structures within the data without any pre-defined label or target variable. 

Unsupervised learning has multiple applications to trading
- clustering 
- dimensionality reduction 
- anomaly detection
- market regime detection 
- portfolio optimization 

Our plan for this project:
 - Download SP500 stocks prices data
 - Calculate different technical indicators and features for each stock 
 - Aggregate monthly and filter only top 150 most liquid stocks for each month
 - Calculate monthly returns for different time horizons 
 - Use Fama-French Factors to calculate rolling betas for each stock 
 - Train a K-Means model for each month to cluster similar stocks together
 - For each month, select assets based on a cluster and form a portfolio using Efficient Frontier max sharpe ratio portfolio optimisation 
 - Visualise portfolio returns and compare with SP500 

=> LIMITATION: We are using only the most recent SP500 stocks list and therefore suffer from the surviviorship bias. We should use surviviorship free data. 

### 1. Downloading Data

In [156]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings("ignore")

sp500 = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]

sp500["Symbol"] = sp500["Symbol"].replace(".", "-")
symbols_list = sp500["Symbol"].unique().tolist()

end_date = "2024-07-09"
start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

df = yf.download(tickers = symbols_list, start=start_date, end=end_date).stack()

df.index.names = ['date','ticker']
df.columns = df.columns.str.lower()

df


[******                13%%                      ]  63 of 502 completed

$BF.B: possibly delisted; No price data found  (1d 2016-07-11 00:00:00 -> 2024-07-09)


[*********************100%%**********************]  502 of 502 completed

2 Failed downloads:
['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2016-07-11 00:00:00 -> 2024-07-09)')


Price               adj close       close  ...        open      volume
date       ticker                          ...                        
2016-07-11 A        42.635677   45.400002  ...   45.610001   1094700.0
           AAL      29.945490   31.160000  ...   30.230000  12374400.0
           AAPL     22.268703   24.245001  ...   24.187500  95179600.0
           ABBV     45.603836   64.349998  ...   64.250000   9641500.0
           ABT      36.353954   42.119999  ...   42.029999   9052300.0
...                       ...         ...  ...         ...         ...
2024-07-08 XYL     134.059998  134.059998  ...  134.809998    746800.0
           YUM     127.940002  127.940002  ...  129.869995   1846100.0
           ZBH     106.389999  106.389999  ...  107.839996   1651100.0
           ZBRA    314.480011  314.480011  ...  311.989990    209100.0
           ZTS     175.429993  175.429993  ...  175.410004   1704700.0

[985886 rows x 6 columns]

### 2. Calculating features and techical indicators for each stock

For each stock, we will compute the following metrics:
 - Garman-Klass volatility
 - RSI
 - Bollinger Bands
 - ATR
 - MACD
 - Dollar volume

In [201]:
df["garman-klass volatility"] = ((np.log(df["high"]) - np.log(df["low"]))**2)/2 - (2*np.log(2)-1)*((np.log(df["adj close"])-np.log(df["open"]))**2)
df["rsi"] = df.groupby(level=1)["adj close"].transform(lambda x: pandas_ta.rsi(close=x,length=20))

# We want to normalise the indicators from this point. We normalise for better clustering.
df["bb_low"] = df.groupby(level=1)["adj close"].transform(lambda x: pandas_ta.bbands(close= np.log1p(x), length = 20).iloc[:,0])
df["bb_mid"] = df.groupby(level=1)["adj close"].transform(lambda x: pandas_ta.bbands(close= np.log1p(x), length = 20).iloc[:,1])
df["bb_high"] = df.groupby(level=1)["adj close"].transform(lambda x: pandas_ta.bbands(close= np.log1p(x), length = 20).iloc[:,2])

def compute_atr(stock_data):
    atr = pandas_ta.atr(high = stock_data["high"], low = stock_data["low"], close = stock_data["close"], length = 14)
    return atr.sub(atr.mean()).div(atr.std())

df["atr"] = df.groupby(level=1, group_keys=False).apply(compute_atr)

def compute_macd(close):
    macd = pandas_ta.macd(close = close, length = 20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df["macd"] = df.groupby(level=1, group_keys=False)["adj close"].apply(compute_macd)

df["dollar_volume"] = (df["close"]*df["volume"])/1e6

df

Price               adj close       close  ...      macd  dollar_volume
date       ticker                          ...                         
2016-07-11 A        42.635677   45.400002  ...       NaN      49.699382
           AAL      29.945490   31.160000  ...       NaN     385.586302
           AAPL     22.268703   24.245001  ...       NaN    2307.629482
           ABBV     45.603836   64.349998  ...       NaN     620.430510
           ABT      36.353954   42.119999  ...       NaN     381.282866
...                       ...         ...  ...       ...            ...
2024-07-08 XYL     134.059998  134.059998  ... -1.070050     100.116006
           YUM     127.940002  127.940002  ... -1.846067     236.190039
           ZBH     106.389999  106.389999  ... -0.868470     175.660528
           ZBRA    314.480011  314.480011  ... -0.004728      65.757770
           ZTS     175.429993  175.429993  ...  0.272868     299.055509

[985886 rows x 14 columns]

### 3. Aggregate on a monthly level and select top 150 most liquid stocks 

We do this aggregation to reduce treining time and therefore facilitate the experimentation with different features and strategies.

In [202]:
# Create a table containing only aggregates of those metrics we calculated + those that can be aggregated. 

last_cols = [c for c in df.columns.unique(0) if c not in ["dollar_volume","volume","open","high","low","close"]]

# dollar_volume is aggregated based on its mean over a month. All others use the value at a month's end 
# as the value for this month.
data = pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack("ticker").to_frame("dollar_volume"), 
df.unstack()[last_cols].resample("M").last().stack("ticker")],axis=1).dropna()

# Calculating dollar_volume 5-year rolling average for each stock before selecting
data["dollar_volume"] = (data.loc[:,"dollar_volume"].unstack("ticker").rolling(5*12,min_periods=12).mean().stack())

data["dollar_volume_rank"] = data.groupby("date")["dollar_volume"].rank(ascending=False)

# Selecting top 150 stocks in terms of dollar volume.
data = data[data["dollar_volume_rank"]<150].drop(["dollar_volume","dollar_volume_rank"],axis=1)

### 4. Calculate monthly returns over different time horizons and add them as features

We do this in order to reflect different time series dynamics which reflect the momentum patterns for each stock. We will compute over 1 month, 2 months, 3 months, 6 months, 9 months, and 12 months

In [203]:
def calculate_returns(df):

    lags = [1,2,3,6,9,12]

    # There will certainly be outliers in our data. If it's the case we want to clip them. We detect outliers 
    # as points in the 99.5 percentile.
    outlier_threshold = 0.005

    for lag in lags:
        df[f"return_{lag}m"] = (df["adj close"]
                                .pct_change(lag)
                                .pipe(lambda x: x.clip(lower = x.quantile(outlier_threshold),upper=x.quantile(1-outlier_threshold)))
                                .add(1)
                                .pow(1/lag)
                                .sub(1))
    
    return df

data = data.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()

data

adj close  garman-klass volatility  ...  return_9m  return_12m
date       ticker                                       ...                       
2018-07-31 AAL      38.702709                 0.000092  ...  -0.017899   -0.019393
           AAPL     45.223690                -0.000885  ...   0.014540    0.022060
           ABBV     70.902939                -0.025348  ...   0.005495    0.026310
           ABT      59.436668                -0.003467  ...   0.022867    0.025837
           ACN     146.102753                -0.003209  ...   0.013581    0.019435
...                       ...                      ...  ...        ...         ...
2024-07-31 XOM     112.180000                 0.000178  ...   0.009436    0.006659
           MRNA    116.449997                 0.000400  ...   0.048618   -0.000861
           UBER     71.250000                 0.000104  ...   0.056952    0.030887
           CRWD    390.709991                 0.000995  ...   0.092124    0.076311
           ABNB    151.960007                 0.000247  ...   0.028222   -0.000126

[10456 rows x 14 columns]

### 5. Download the Fama-French factors and compute rolling factors betas

 - We will introduce the Fama-French to compute the exposure the exposure of assets to common risk factors using linear regression. 
 - The 5 Fama-French factors are:
    - market risk
    - size
    - value 
    - operating profitability
    - investment 

=> They have been shown to explain asset returns and are commonly used to assess the risk/return profile of portfolios. 

In [204]:
# Downloading Fama-French factors.
# For the scope of this project, we are ignoing risk-free rate. This should 
# not be a done in a strategy that is intended to be implemented. 
factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3',
               "famafrench",
               start = "2010")[0].drop("Mkt-RF",axis=1)

factor_data.index = factor_data.index.to_timestamp()

# Aligns the factors with our data that is recorded for the end of a month and not the beginning. 
# The data is expressed as a %, we want it as a plain decimal and therefore convert. 
factor_data = factor_data.resample("M").last().div(100)
factor_data.index.name = 'date'

"""
We associate the Fama-French factors which are said to be predictive with the return over 1 month and will conduct 
linear regression to compute beta.
"""
factor_data = factor_data.join(data["return_1m"]).sort_index()
# filter_out stocks with less than 10 monhts of data
observations = factor_data.groupby(level=1).size()
valid_stocks = observations[observations >= 10]

factor_data = factor_data[factor_data.index.get_level_values("ticker").isin(valid_stocks.index)]

# Computing rolling factor betas

betas = (factor_data.groupby(level=1,group_keys=False)
 .apply(lambda x: RollingOLS(endog=x["return_1m"],exog=sm.add_constant(x.drop("return_1m",axis=1)), window= min(24, x.shape[0]), min_nobs=len(x.columns)+ 1)
        .fit(params_only=True).params.drop("const",axis=1)))


factors = ["SMB","HML","RMW","CMA","RF"]

data = data.join(betas.groupby("ticker").shift())
# Inputing all unknown values from the factor's average.
data.loc[:,factors] = data.groupby('ticker',group_keys=False)[factors].apply(lambda x: x.fillna(x.mean()))

final_data = data.dropna()

data = data.drop("adj close", axis = 1)

data.info()

""" 
Our final dataframe has structure:
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   garman-klass volatility  10456 non-null  float64
 1   rsi                      10456 non-null  float64
 2   bb_low                   10456 non-null  float64
 3   bb_mid                   10456 non-null  float64
 4   bb_high                  10456 non-null  float64
 5   atr                      10456 non-null  float64
 6   macd                     10456 non-null  float64
 7   return_1m                10456 non-null  float64
 8   return_2m                10456 non-null  float64
 9   return_3m                10456 non-null  float64
 10  return_6m                10456 non-null  float64
 11  return_9m                10456 non-null  float64
 12  return_12m               10456 non-null  float64
 13  SMB                      10166 non-null  float64
 14  HML                      10166 non-null  float64
 15  RMW                      10166 non-null  float64
 16  CMA                      10166 non-null  float64
 17  RF                       10166 non-null  float64
dtypes: float64(18)
"""


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 10456 entries, (Timestamp('2018-07-31 00:00:00'), 'AAL') to (Timestamp('2024-07-31 00:00:00'), 'ABNB')
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   garman-klass volatility  10456 non-null  float64
 1   rsi                      10456 non-null  float64
 2   bb_low                   10456 non-null  float64
 3   bb_mid                   10456 non-null  float64
 4   bb_high                  10456 non-null  float64
 5   atr                      10456 non-null  float64
 6   macd                     10456 non-null  float64
 7   return_1m                10456 non-null  float64
 8   return_2m                10456 non-null  float64
 9   return_3m                10456 non-null  float64
 10  return_6m                10456 non-null  float64
 11  return_9m                10456 non-null  float64
 12  return_12m               10456 non-null  float64
 13 

' \nOur final dataframe has structure:\n #   Column                   Non-Null Count  Dtype  \n---  ------                   --------------  -----  \n 0   garman-klass volatility  10456 non-null  float64\n 1   rsi                      10456 non-null  float64\n 2   bb_low                   10456 non-null  float64\n 3   bb_mid                   10456 non-null  float64\n 4   bb_high                  10456 non-null  float64\n 5   atr                      10456 non-null  float64\n 6   macd                     10456 non-null  float64\n 7   return_1m                10456 non-null  float64\n 8   return_2m                10456 non-null  float64\n 9   return_3m                10456 non-null  float64\n 10  return_6m                10456 non-null  float64\n 11  return_9m                10456 non-null  float64\n 12  return_12m               10456 non-null  float64\n 13  SMB                      10166 non-null  float64\n 14  HML                      10166 non-null  float64\n 15  RMW                 

### 6. Clustering our stocks using KMeans Clustering

In [205]:
from sklearn.cluster import  KMeans

# We intialise cluster centroids to have specific RSI values. This enables us to easily interpret the cluster 
# number across all months. 
target_rsi_values = [70, 55, 45, 70]
initial_centroids = np.zeros((len(target_rsi_values),18))
initial_centroids[:,6] = target_rsi_values

def get_clusters(df):
    df["cluster"] = KMeans(n_clusters=4, random_state=0, init=initial_centroids).fit(df).labels_

    return df

data = data.dropna().groupby('date', group_keys =False).apply(get_clusters)

data

garman-klass volatility        rsi  ...         RF  cluster
date       ticker                                      ...                    
2018-07-31 AAL                    0.000092  49.117680  ...  -6.382569        3
           AAPL                  -0.000885  52.605260  ... -10.609648        3
           ABBV                  -0.025348  45.920948  ...   0.324047        2
           ABT                   -0.003467  62.882440  ...   8.036811        2
           ACN                   -0.003209  42.887823  ...  -5.460538        3
...                                    ...        ...  ...        ...      ...
2024-07-31 WMT                    0.000060  66.994252  ...  10.753530        2
           XOM                    0.000178  45.882744  ... -41.292562        3
           MRNA                   0.000400  39.218211  ...   1.174208        2
           UBER                   0.000104  54.411955  ...  27.619386        1
           CRWD                   0.000995  62.038255  ...   5.041030        2

[10166 rows x 19 columns]

### 7. For each month, select a set of assets based on the cluster and form a portfolio based on the Efficient Frontier max sharpe ratio optimization

In [217]:
# We select stocks from cluter 3 which are those with an RSI closer to 70 as our assumption is that momentum will keep on forward.
filtered_df = data[data['cluster'] == 3].copy()
filtered_df = filtered_df.reset_index(level=1)
filtered_df.index = filtered_df.index+pd.DateOffset(1)
filtered_df = filtered_df.reset_index().set_index(['date', 'ticker'])

dates = filtered_df.index.get_level_values('date').unique().tolist()
fixed_dates = {}
for d in dates:
    fixed_dates[d.strftime("%Y-%m-%d")] = filtered_df.xs(d, level=0).index.tolist()

fixed_dates

{'2018-08-01': ['AAL',
  'AAPL',
  'ACN',
  'ADBE',
  'AIG',
  'AMAT',
  'AXP',
  'BA',
  'BAC',
  'BIIB',
  'CL',
  'CMG',
  'COST',
  'CRM',
  'CSCO',
  'CVS',
  'DD',
  'DHR',
  'DUK',
  'FCX',
  'GOOG',
  'GOOGL',
  'GS',
  'HD',
  'HUM',
  'INTC',
  'ISRG',
  'KHC',
  'KR',
  'LLY',
  'LMT',
  'LOW',
  'MAR',
  'MET',
  'MPC',
  'MS',
  'MSFT',
  'MU',
  'NFLX',
  'NKE',
  'ORCL',
  'PFE',
  'PNC',
  'QCOM',
  'REGN',
  'RTX',
  'SCHW',
  'SLB',
  'STZ',
  'TGT',
  'TMO',
  'TSLA',
  'UNP',
  'VLO'],
 '2018-09-01': ['AAL',
  'AAPL',
  'ACN',
  'ADBE',
  'AIG',
  'AMAT',
  'AMZN',
  'AXP',
  'BA',
  'BAC',
  'BIIB',
  'CL',
  'CMG',
  'COST',
  'CRM',
  'CSCO',
  'CVS',
  'CVX',
  'DD',
  'DHR',
  'DUK',
  'EBAY',
  'F',
  'FCX',
  'GOOG',
  'GOOGL',
  'GS',
  'HD',
  'HUM',
  'ISRG',
  'KHC',
  'KR',
  'LLY',
  'LMT',
  'LOW',
  'MET',
  'MPC',
  'MSFT',
  'MU',
  'NFLX',
  'NKE',
  'ORCL',
  'PFE',
  'PNC',
  'QCOM',
  'REGN',
  'RTX',
  'SCHW',
  'SLB',
  'STZ',
  'TGT',
  'TMO'

In [234]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models, expected_returns

# Providing this function with price data for each stock for the past year will lead to weights forming a portfolio
# of these stocks with max sharpe ratio.
def optimize_weights(prices,lower_bound = 0):
    returns = expected_returns.mean_historical_return(prices=prices, frequency=252)
    cov = risk_models.sample_cov(prices=prices,frequency=252)
    # We set the max weight to be 10% so that we ensure a diversified portfolio. 
    ef = EfficientFrontier(expected_returns=returns, cov_matrix=cov,solver="SCS",weight_bounds=(lower_bound,.1))
    weights = ef.max_sharpe()
    # clean weights rounds the weights and clips them near 0.
    return ef.clean_weights()

stocks = data.index.get_level_values('ticker').unique().tolist()
new_df = yf.download(tickers = stocks, start = data.index.get_level_values('date').unique()[0]-pd.DateOffset(month=12),
                     end = data.index.get_level_values('date').unique()[-1])
new_df

[*********************100%%**********************]  155 of 155 completed


Price       Adj Close                          ...    Volume                   
Ticker            AAL        AAPL        ABBV  ...       WMT     WYNN       XOM
Date                                           ...                             
2018-12-31  31.599047   37.750694   71.630341  ...  21017400  2609800  15807000
2019-01-02  31.963161   37.793781   69.330475  ...  24458100  4174400  16727200
2019-01-03  29.581665   34.029243   67.046135  ...  24831900  2885100  13866100
2019-01-04  31.530161   35.481915   69.206146  ...  24087300  3007200  16043600
2019-01-07  32.425678   35.402946   70.216225  ...  23369100  3143800  10844200
...               ...         ...         ...  ...       ...      ...       ...
2024-07-02  11.050000  220.270004  165.960007  ...   9436500  2233300  13326700
2024-07-03  11.120000  221.550003  163.839996  ...   6287500  1556900   7732900
2024-07-05  11.040000  226.339996  167.250000  ...  21586100  1925200  12631000
2024-07-08  10.930000  227.820007  166.520004  ...  12073900  2237600  13342200
2024-07-09  11.005000  227.751007  166.960007  ...   2740415   543126   4068021

[1389 rows x 930 columns]

- calculate daily return for each stock from the fresh data
- then loop over each month start, select the stocks for the month and calculate their weights for the next month
- if the maximum sharpe ratio fails for a given month, apply equally-weighted weights
- calculate daily portfolio returns 

In [240]:
returns_dataframe = np.log(new_df['Adj Close']).diff()



portfolio_df = pd.DataFrame()

for start_date in fixed_dates.keys():
    
    # try:

        end_date = (pd.to_datetime(start_date)+pd.offsets.MonthEnd(0)).strftime('%Y-%m-%d')

        cols = fixed_dates[start_date]

        optimization_start_date = (pd.to_datetime(start_date)-pd.DateOffset(months=12)).strftime('%Y-%m-%d')

        optimization_end_date = (pd.to_datetime(start_date)-pd.DateOffset(days=1)).strftime('%Y-%m-%d')
        
        optimization_df = new_df[optimization_start_date:optimization_end_date]['Adj Close'][cols]
        
        success = False
        try:
            weights = optimize_weights(prices=optimization_df,
                                   lower_bound=round(1/(len(optimization_df.columns)*2),3))

            weights = pd.DataFrame(weights, index=pd.Series(0))
            
            success = True
        except:
            print(f'Max Sharpe Optimization failed for {start_date}, Continuing with Equal-Weights')
        
        if success==False:
            weights = pd.DataFrame([1/len(optimization_df.columns) for i in range(len(optimization_df.columns))],
                                     index=optimization_df.columns.tolist(),
                                     columns=pd.Series(0)).T
        
        temp_df = returns_dataframe[start_date:end_date]

        temp_df = temp_df.stack().to_frame('return').reset_index(level=0)\
                   .merge(weights.stack().to_frame('weight').reset_index(level=0, drop=True),
                          left_index=True,
                          right_index=True)\
                   .reset_index().set_index(['Date', 'index']).unstack().stack()

        temp_df.index.names = ['date', 'ticker']

        temp_df['weighted_return'] = temp_df['return']*temp_df['weight']

        temp_df = temp_df.groupby(level=0)['weighted_return'].sum().to_frame('Strategy Return')

        portfolio_df = pd.concat([portfolio_df, temp_df], axis=0)
    
    # except Exception as e:
    #     print(e)

portfolio_df = portfolio_df.drop_duplicates()

portfolio_df

Max Sharpe Optimization failed for 2018-08-01, Continuing with Equal-Weights
"None of ['index'] are in the columns"
Max Sharpe Optimization failed for 2018-09-01, Continuing with Equal-Weights
"None of ['index'] are in the columns"
Max Sharpe Optimization failed for 2018-10-01, Continuing with Equal-Weights
"None of ['index'] are in the columns"
Max Sharpe Optimization failed for 2018-11-01, Continuing with Equal-Weights
"None of ['index'] are in the columns"
Max Sharpe Optimization failed for 2018-12-01, Continuing with Equal-Weights
"None of ['index'] are in the columns"
Max Sharpe Optimization failed for 2019-01-01, Continuing with Equal-Weights
"None of ['index'] are in the columns"
"None of ['index'] are in the columns"
"None of ['index'] are in the columns"
"None of ['index'] are in the columns"
"None of ['index'] are in the columns"
"None of ['index'] are in the columns"
"None of ['index'] are in the columns"
"None of ['index'] are in the columns"
"None of ['index'] are in the c

""
